### Heap-Hashmap Architecture

Note: since heap is not a stable sorting algorithm, we must order each element by both price and a timestamp. We also are not storing quantity in the heap, so the unique price and timestamp tuple must be some key in a hashmap storing the quantity available at that price. 

Currently I am not passing an order ID, nor an AuthorID for each order. In order to properly log the trades, we will need unique IDs for each order, each trade, and we will need to store all of those placing orders in another data structure, probably a hashmap, with clientID as the key and their actual alias as the value.

However, if we store only each available price level in the heap, we can maintain a priority queue associated with each price level, which will support linear amortized-time order book matching assuming there are an average of more than log(# of price levels) orders per price level.

In [3]:
#matching engine... should be queried before placing an order into the book. So it's not added if not necessary
#matching engine will use the current order, and check the book for orders that it could be paired to. If 
#pair-able, fill pair-able orders until no more (either fully filled, or partial fill with no more available at
#a pair-able price leve).

#for efficiency, we want to query all orders at a matchable price level, in price-time priority, such that we can
#just loop through those until done with the fill

#matching engines store the order book in memory.. so the book class should contain the me.. or really the ME 
#class should contain the book..

import heapq
import pandas as pd

class matching_engine:
  def __init__(self):
    self.bids = []
    self.offers = []
    self.qtys = {}

  def add_bid(self, price, qty, timestamp):
    if self.offers and self.match_bid(price, qty, timestamp):
      return
    heapq.heappush(self.bids, (-price, timestamp))
    self.qtys[(-price, timestamp)] = qty

  def add_offer(self, price, qty, timestamp):
    if self.bids and self.match_offer(price, qty, timestamp):
      return
    heapq.heappush(self.offers, (price, timestamp))
    self.qtys[(price, timestamp)] = qty

  def match_bid(self, price, qty, timestamp):
    min_offer = self.offers[0]
    if price >= min_offer[0]:
      min_offer_qty = self.qtys[min_offer]
      trade_size = min(min_offer_qty, qty)
      qty -= trade_size
      self.qtys[min_offer] -= trade_size

      print(f"Trade Executed: size = {trade_size} \t price = {min_offer[0]}")

      if not self.qtys[min_offer]:
        heapq.heappop(self.offers)
        del self.qtys[min_offer]
      if qty:
        self.add_bid(price, qty, timestamp)
      return True

    return False
    
  def match_offer(self, price, qty, timestamp):
    min_bid = self.bids[0]
    if price <= -min_bid[0]:
      min_bid_qty = self.qtys[min_bid]
      trade_size = min(min_bid_qty, qty)
      qty -= trade_size
      self.qtys[min_bid] -= trade_size

      print(f"Trade Executed: size = {trade_size} \t price = {-min_bid[0]}")

      if not self.qtys[min_bid]:
        heapq.heappop(self.bids)
        del self.qtys[min_bid]
      if qty:
        self.add_offer(price, qty, timestamp)
      return True

    return False
  
  def validate(self, o_type, price, qty, timestamp):
    valid = True
    if o_type not in 'bBoO':
      print(f"Error Log: Order Validation - Order Type Error: \nOrder [\'{o_type}\', {price}, {qty}, {timestamp}]\n\
Invalid order type: \'{o_type}\'.\nValid Orders are \'b\' or \'o\'.")
      valid = False
    if price <= 0:
      print(f"Error Log: Order Validation - Price Specification Error: \nOrder [\'{o_type}\', {price}, {qty}, {timestamp}]\n\
Invalid Price: {price}.\nValid Prices are Greater than Zero (p > 0)")
      valid = False
    if qty <= 0:
      print(f"Error Log: Order Validation - Size Specification Error: \nOrder [\'{o_type}\', {price}, {qty}, {timestamp}]\n\
Invalid Size: {price}.\nValid Sizes are Greater than Zero (qty > 0)")
      valid = False
    if timestamp <= 0:
      print(f"Error Log: Order Validation - Timestamp Specification Error: \nOrder [\'{o_type}\', {price}, {qty}, {timestamp}]\n\
Invalid Size: {price}.\nValid Timestamps are Greater than Zero (t > 0)")
      valid = False
    return valid

  def new_order(self, o_type, price, qty, timestamp):
#add function to log orders appropriately
    if not self.validate(o_type, price, qty, timestamp):
      return
    if o_type in 'bB':
      self.add_bid(price, qty, timestamp)
    else:
      self.add_offer(price, qty, timestamp)
    return

  def get_book(self):
    bids_copy = self.bids.copy()
    offers_copy = self.offers.copy()
    rows = []

    while bids_copy or offers_copy:
      cur_row = []
      if bids_copy:
        b_p, b_t = heapq.heappop(bids_copy)
        b_qty = self.qtys[(b_p, b_t)]
        cur_row.append(str(b_qty))
        cur_row.append(str(-b_p))
      else:
        cur_row.append('')
        cur_row.append('')
      
      if offers_copy:
        o_p, o_t = heapq.heappop(offers_copy)
        o_qty = self.qtys[(o_p, o_t)]
        cur_row.append(o_p)
        cur_row.append(o_qty)
      else:
        cur_row.append('')
        cur_row.append('')
      rows.append(cur_row)

    book = pd.DataFrame(columns=['bid_qty', 'bid_price', 'offer_price', 'offer_qty'], data=rows)

    display(book)
    return

ME = matching_engine()

ME.new_order('o', 12, 100, 1)
ME.new_order('b', 11.50, 50, 2)
ME.new_order('b', 11.75, 100, 3)
ME.new_order('b', 11.5, 25, 4)
ME.new_order('o', 12.05, 75, 5)
ME.new_order('o', 11.50, 80, 6)
ME.get_book()

Trade Executed: size = 80 	 price = 11.75


,bid_qty,bid_price,offer_price,offer_qty
0,20,11.75,12,100
1,50,11.5,12.05,75
2,25,11.5,,


### Heap-Hashmap-Priority Queue Architecture

by maintaining a priority queue to sort orders at the same price level, we don't even have to add timestamps. The priority queue is stable. In our heap, all we have to store is the same as the set of all keys in the hashmap. Each key corresponds to a price level that has at least an active order. Each unique order in the priority queue will have to contain a corresponding orderID, and quantity. Each element can be a len() 2 list containing (1) the order ID, and (2) the remaining quantity of that trade. All trades are assumed to be active until filled. Once we have a unique order ID for each order, we can store those IDs in their own DB (a map) to access them in linear time, allowing us to remove_order(order_ID) efficiently.